# Basics of QuantLib Python

QuantLib is a quantitative finance library available in Python language. This notebook introduces the basic classes and functionality in QL.

In [1]:
import QuantLib as ql

#### Time SubModule

The `ql/time` subfolder implements various time-related classes. Let's take a look at the `Date` object which can be constructed as `Date(date, month, year)`.

In [2]:
date = ql.Date(1, 3, 2021) # 1 March 2021
print(date)

March 1st, 2021


In [3]:
date.dayOfMonth()

1

In [4]:
date.month()

3

In [5]:
date.year()

2021

In [7]:
date.weekday() == ql.Monday

True

In [8]:
# arithmetic with dates
date + 1 # add a day

Date(2,3,2021)

In [9]:
date - 1 # subtract a day

Date(28,2,2021)

In [10]:
date + ql.Period(1, ql.Months) # add a month

Date(1,4,2021)

In [11]:
date + ql.Period(1, ql.Years) # add a year

Date(1,3,2022)

In [12]:
# logical operations
ql.Date(1,3, 2021) > ql.Date(31, 3, 2021)

False

The `Schedule` object can be used to construct a list of dates such as coupon payments. Let's look at some examples.

In [13]:
date1 = ql.Date(1, 1, 2021)
date2 = ql.Date(1, 1, 2022)
tenor = ql.Period(ql.Monthly)
calendar = ql.UnitedStates()
schedule = ql.Schedule(date1, date2, tenor, calendar, ql.Following, 
                      ql.Following, ql.DateGeneration.Forward, False)
list(schedule)

[Date(4,1,2021),
 Date(1,2,2021),
 Date(1,3,2021),
 Date(1,4,2021),
 Date(3,5,2021),
 Date(1,6,2021),
 Date(1,7,2021),
 Date(2,8,2021),
 Date(1,9,2021),
 Date(1,10,2021),
 Date(1,11,2021),
 Date(1,12,2021),
 Date(3,1,2022)]

Here we've generated a `Schedule` object that will contain dates between `date1` and `date2` with the `tenor` specifying the `Period` to be every month. The `calendar` object is used for determining holidays. The two arguments following the `calendar` in the `Schedule` constructor are the `BusinessDayConvention`. Here we chose the convention to be the day following holidays. That is why we see that holidays are excluded in the list of dates.

#### Interest Rate

The `InterestRate` class can be used to store the interest rate with the compounding type, day count and the frequency of compounding. Below we show how to create an interest rate of 5.0% compounded annually, using Actual/Actual day count convention.

In [14]:
annualRate = 0.05
dayCount = ql.ActualActual()
compoundType = ql.Compounded
frequency = ql.Annual
interestRate = ql.InterestRate(annualRate, dayCount, compoundType,
                              frequency)

Let's say if you invest a dollar at the interest rate described by `interestRate`, the `compoundFactor` method gives you how much your investment will be worth after `t` years. Below we show that the value returned by `compoundFactor` for 2 years agrees with the expected compounding formula.

In [15]:
interestRate.compoundFactor(2.0)

1.1025

In [16]:
# verification
(1.0 + annualRate)*(1.0 + annualRate)

1.1025

The `discountFactor` method returns the reciprocal of the `compoundFactor` method. The discount factor is useful while calculating the present value of future cashflows.

In [17]:
interestRate.discountFactor(2.0)

0.9070294784580498

In [18]:
1.0/interestRate.compoundFactor(2.0)

0.9070294784580498

A given interest rate can be converted into other types using the `equivalentRate` method as:

In [19]:
newFrequency = ql.Semiannual
effectiveRate = interestRate.equivalentRate(compoundType, newFrequency, 1)
effectiveRate.rate()

0.04939015319191986

The `InterestRate` class also has an `impliedRate` method. The `impliedRate` method takes compound factor to return the implied rate. The `impliedRate` method is a static method in the `InterestRate` class and can be used without an instance of `InterestRate`. Internally the `equivalentRate` method invokes the `impliedRate` method in its calculations.

Here we have converted into a semi-annual compounding type. A 4.939% of semi-annual compounding is equivalent to 5.0% annual compounding. This should mean that both should give identical discount factors:

In [20]:
interestRate.discountFactor(1.0)

0.9523809523809523

In [21]:
effectiveRate.discountFactor(1.0)

0.9523809523809521